### Import modules

In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import pandas as pd
import matplotlib.pyplot as plt
import warnings

# ignore the overflow warnings
np.seterr(over='ignore')

#ignore the runtime warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

### Read the train.csv

In [2]:
with open("train.csv", "r", encoding='latin1') as train:
    csvfile = pd.read_csv(train, header=None)

csvfile = csvfile.values.tolist()

labeltxt = []
smstxt = []

# store the label and the sms into two seperated list
for line in csvfile:
    labeltxt.append(line[0])
    smstxt.append(line[1])

labelnum = np.array([])

# convert the 'ham' into 1, and 'spam' into 0
for i in labeltxt:
    if i == 'ham':
        labelnum = np.append(labelnum, 1)
    if i == 'spam':
        labelnum = np.append(labelnum, 0)

vectorizer = CountVectorizer()
smsnum = vectorizer.fit_transform(smstxt).toarray()

# ti-idf
transformer = TfidfTransformer(smooth_idf=True)
smstf = transformer.fit_transform(smsnum).toarray()

print('the shape of vectorized sms array set is: ',smstf.shape)

the shape of vectorized sms array set is:  (3000, 6277)


### Generate a random value w0 from range(-1,6), as the initial weights
since the loss function is convex and the weights w is unconstrained, I can start anywhere I want.

In [3]:
w0 = np.random.randint(-1, 7, 6277, dtype=int)

### Define computing functions of the batch gradient descent
for all the functions:
y is the binary labels,
x is the vectorized sms ndarray after tf-idf,
w is the weight,
lam is the lambda, the coefficient of the regularizer.

In [4]:
# sigmoid function
def sigmoid(w, x):
    return 1/(1+np.exp(-(np.dot(x, w)), dtype=np.float128))

# the derivative of sigmoid function respect to w
def gradient_sigmoid(w, x):
    return ((1/(2+np.exp(np.dot(x, w), dtype=np.float128) 
                + np.exp(-np.dot(x, w), dtype=np.float128))).reshape(len(x), 1))*x


# the loss function with regularizer
def loss(y, x, w, lam):
    loss_compute = ((-y)*(np.log(sigmoid(w, x), dtype=np.float128))) \
                   - ((1-y)*(np.log(1-sigmoid(w, x), dtype=np.float128))) \
                   + (lam*np.power(np.linalg.norm(w), 2))
    return sum(loss_compute)

# the gradient of the loss function respect to w
def gradient(y, x, w, lam):
    y_col = y.reshape(1, len(y))
    exp = (np.exp(-np.dot(x, w))).reshape(1, len(y))
    g = np.dot((-y_col)*(1+exp), gradient_sigmoid(w, x)) + np.dot((1-y_col)*((1+exp)/exp), gradient_sigmoid(w, x))
    g = (g.reshape(len(w),)) + 2*lam*w
    return g

# the batch gradient descent algorithm function 
# I set a fixed learning rate 0.3. 
def gradient_descent(w, x, y, lam):
    w_old = w
    step = 0.3
    loss_old = loss(y, x, w_old, lam)
    w_new = w_old - step*gradient(y, x, w_old, lam)
    loss_new = loss(y, x, w_new, lam)

    if loss_new - loss_old >= 0:
        return w_old
    else:
        return gradient_descent(w_new, x, y, lam)

### 10-folds cross validation

In [5]:
# random shuffle all the emails and divide into 10 groups
index = np.arange(3000)
np.random.shuffle(index)
index = index.reshape(10, 300)

# initial several arrays for future use, to store the data
test = np.zeros((300, 6277), dtype=np.float128)
train = np.zeros((2700, 6277), dtype=np.float128)

test_label = np.zeros(300, dtype=int)
train_label = np.zeros(2700, dtype=int)

lamda = np.array([])
rate_test = np.array([])
rate_train = np.array([])

# iteration for the different lambda
# the range of lambda I set is from (0.01, 0.2), with step 0.01
print('computing start!')
for i in range(0, 20):
    lamda = np.append(lamda, 0.01+i*0.01)
    rate_temp_test = 0
    rate_temp_train = 0
    
    # 10-folds cross validation
    for k in range(0, 10):
        test_index = index[k]
        train_index = np.delete(index, k, 0).reshape(2700)
        
        # construct the test set, with size of 300
        ct = 0
        for n in test_index:
            test[ct] = smstf[n]
            test_label[ct] = labelnum[n]
            ct = ct + 1
        
        # construct the train set, with size of 2700
        ct = 0
        for n in train_index:
            train[ct] = smstf[n]
            train_label[ct] = labelnum[n]
            ct = ct + 1
        # implement the gradient descent algorithm to compute the weight
        w0 = gradient_descent(w0, train, train_label, lamda[i])

        # compute accuracy on test set
        hit = 0
        j = 0
        for sms in test:
            result = sigmoid(w0, sms)
            if result >= 0.5:
                result = 1
            else:
                result = 0

            if result == test_label[j]:
                hit = hit + 1
            j = j + 1

        rate_temp_test = rate_temp_test + hit/300

        # compute accuracy on train set
        hit = 0
        j = 0
        for sms in train:
            result = sigmoid(w0, sms)
            if result >= 0.5:
                result = 1
            else:
                result = 0

            if result == train_label[j]:
                hit = hit + 1
            j = j + 1

        rate_temp_train = rate_temp_train + hit/2700

    rate_test = np.append(rate_test, 100*(rate_temp_test/10))
    rate_train = np.append(rate_train, 100*(rate_temp_train/10))
    print('computing......',5*(i+1),'% complete')

print('Done!')

computing start!


RecursionError: maximum recursion depth exceeded while calling a Python object

### Result analysis

In [ ]:
print('optimal average accuracy rate on test: ', np.amax(rate_test),'percentage')
lamda_opt = lamda[np.argmax(rate_test)]
print('optimal lambda is：', lamda_opt)

# compute the optimal weight using the optimal lambda
w_opt = gradient_descent(w0, smstf, labelnum, lamda_opt)
# np.savez('w.npz', w_opt=w_opt)

print('optimal loss is: ', loss(labelnum, smstf, w_opt, lamda_opt))

# plot the curve of lambda versus accuracy
plt.plot(lamda, rate_train, 'b', label='trainset accuracy')
plt.plot(lamda, rate_test, 'r', label='testset accuracy')

plt.ylabel('accuracy rate (percentage)')
plt.xlabel('lambda')
plt.legend()
plt.show()


### Model test
Read the test.csv dataset, then use the optimal model w_opt to do test. Report the accuracy rate.

In [ ]:
with open("test.csv", "r", encoding='latin1') as test:
    testfile = pd.read_csv(test, header=None)

testfile = testfile.values.tolist()

labeltest = []
smstest = []

# store the label and the sms into two seperated list
for line in testfile:
    labeltest.append(line[0])
    smstest.append(line[1])

labeltestnum = np.array([])

# convert the 'ham' into 1, and 'spam' into 0
for i in labeltest:
    if i == 'ham':
        labeltestnum = np.append(labeltestnum, 1)
    if i == 'spam':
        labeltestnum = np.append(labeltestnum, 0)

transformer = TfidfTransformer(smooth_idf=True)

ct = 0
i = 0

# count the number of right match.
for sms in smstest:
    
    # apply the same dict to vectorize the test set
    testarray = vectorizer.transform([sms]).toarray()
    
    # tf_idf transform
    testarray = transformer.fit_transform(testarray).toarray()
    
    # compute the result of sigmoid function, using w_opt
    result = sigmoid(w_opt, testarray)
    
    if result >= 0.5:
        result = 1
    else:
        result = 0

    if result == labeltestnum[i]:
        ct = ct + 1
    
    i = i+1

print('the number of right match is:',ct)
print('the accuracy rate is:',100*ct/len(labeltestnum), 'percentage')